## Markov chains

We'll cover the following topics in this notebook:
- Random number generation
- Markov chains: definition, stationary distribution 
- Modelling Markov chains


### Random number generation

In [1]:
(* 
	OCaml's Random module provides a pseudo-random number generator.
	The term "pseudo" means that the number it generates 
	depends on a random "seed" given at initialization.
	The same seed always generates the same "random" sequence. 
*)

(* init : int -> unit *)
(* Initializes the RNG with seed. *)

let seed = 0
let () = Random.init seed

(* int : int -> int *)
(* [Random.int bound] Generates an integer i such that 0 <= i < bound. *)

let _ = Random.int 10
let ls = List.init 10 (fun _ -> Random.int 10)

(* Simple simulation *)
let compute_prob total =
	let counts = Array.make 10 0. in
	let samples = List.init total (fun _ -> Random.int 10) in
	let rec counting = function
		| [] -> ()
		| x :: xs -> 
			counts.(x) <- counts.(x) +. 1.;
			counting xs
	in 
		counting samples;
		Array.map (fun x -> x /. (float_of_int total)) counts

val seed : int = 0


- : int = 2


val ls : int list = [0; 4; 6; 2; 3; 8; 2; 4; 7; 8]


val compute_prob : int -> float array = <fun>


In [2]:
(*
	Markov chains

	We now build a probabilistic model: a Markov chain,
	which is like a finite state machine, but the state
	transition happens randomly, and the probability of
	reaching the next state only depends on the
	current state. 
*)

type states = A | B

let next_state current = 
	let p = Random.float 1.0 in
	match current with
		| A -> if p <= 0.5 then A else B
		| B -> if p <= 0.5 then A else B

let trace init steps = 
		let current = ref init in
		let walk () = 
			current := next_state (!current);
			!current
		in
			List.init steps (fun _ -> walk())

(* 
	We can sample and estimate the stationary distribution 
	of our Markov chain: the proportion of each state's 
	appearance in an infinite sequence.
*)

let dist init steps = 
	let counts = [|0.; 0.|] in
	let tr = trace init steps in
	let rec aux = function
		| [] -> ()
		| A :: xs -> counts.(0) <- counts.(0) +. 1.; aux xs
		| B :: xs -> counts.(1) <- counts.(1) +. 1.; aux xs
	in
		aux tr;
		Array.map (fun x -> x /. (float_of_int steps)) counts

type states = A | B


val next_state : states -> states = <fun>


val trace : states -> int -> states list = <fun>


val dist : states -> int -> float array = <fun>


In [3]:
(*
	Puzzle: In a game, the main character's attack has 
	25% chance of being a critical hit. To avoid getting
	long sequences without a critical, the developers
	used a "fake-random" algorithm that guarantees
	a critical in every four hits. What is the actual
	probability of getting a critical hit?
*)

type states = S0 | S1 | S2 | Crit

let next_state current = 
	let p = Random.int 4 in
	match current with
		| S0 -> if p = 3 then S0 else S1 
		| S1 -> if p = 3 then S0 else S2
		| S2 -> if p = 3 then S0 else Crit
		| Crit -> S0

let trace init steps = 
		let current = ref init in
		let walk () = 
			current := next_state (!current);
			!current
		in
			List.init steps (fun _ -> walk())
	
let dist init steps = 
	let counts = [|0.; 0.; 0.; 0.|] in
	let tr = trace init steps in
    let steps = float_of_int steps in
	let rec aux = function
		| [] -> ()
		| S0 :: xs -> counts.(0) <- counts.(0) +. 1.; aux xs
		| S1 :: xs -> counts.(1) <- counts.(1) +. 1.; aux xs
		| S2 :: xs -> counts.(2) <- counts.(2) +. 1.; aux xs
		| Crit :: xs -> counts.(3) <- counts.(3) +. 1.; aux xs
	in
		aux tr;
		Array.map (fun x -> x /. steps) counts 

let actual = Array.map (fun x -> x /. 175.) [|64.; 48.; 36.; 27. |]

type states = S0 | S1 | S2 | Crit


val next_state : states -> states = <fun>


val trace : states -> int -> states list = <fun>


val dist : states -> int -> float array = <fun>


val actual : float array =
  [|0.365714285714285714; 0.274285714285714299; 0.205714285714285711;
    0.154285714285714276|]
